In [1]:
import plotly.io as pio

pio.renderers.default = "plotly_mimetype+notebook_connected"

import pandas as pd
import pycountry
from pycountry_convert import country_alpha2_to_continent_code, convert_continent_code_to_continent_name

# Function to get the continent of each country
def iso_to_continent(country_iso):
    try:
        country = pycountry.countries.get(alpha_3=country_iso)
        country_alpha2 = country.alpha_2
        continent_code = country_alpha2_to_continent_code(country_alpha2)
        continent_name = convert_continent_code_to_continent_name(continent_code)
        return continent_name
    except (KeyError, AttributeError):
        return None

# Clean dataset    
df = pd.read_csv("population1.csv")
df = df.dropna(subset=['Country of origin (ISO)', 'Country of asylum (ISO)'])
df['asy_continent'] = df['Country of asylum (ISO)'].apply(iso_to_continent)
df = df.dropna()
df2 =df[df["Country of origin (ISO)"]=="UKR"]
df2 = df2[df2["Year"] >= 2014]

# Combine minor countries as other
df3 = df2[df2["Country of asylum"].isin(["Russian Federation", "Poland", "Germany", "Czechia"])]
df4 = df2[~df2["Country of asylum"].isin(["Russian Federation", "Poland", "Germany", "Czechia"])]
df5 = df4.groupby("Year").sum().reset_index()
df5["Country of asylum"] = "Other"
df6 = pd.concat([df3[["Year", "Country of asylum", "Refugees under UNHCR's mandate"]], df5[["Year", "Country of asylum", "Refugees under UNHCR's mandate"]]])

# Plot with plotly
import plotly.express as px
fig = px.area(df6, x="Year", y="Refugees under UNHCR's mandate", color="Country of asylum", template="simple_white")
fig.update_layout(title_text="Asylum Countries of Ukraine Refugees from 2014 to 2022",template="plotly_white",legend=dict(
    orientation="h", y=-0.2
),     title=dict(
    text="Asylum Countries of Ukraine Refugees from 2014 to 2022",
    font=dict(
    family="Arial", size=20)
    ),

    yaxis_title=dict(
    font=dict(
    family="Arial", size=18)
    ),

    xaxis_title=dict(
    font=dict(
    family="Arial", size=18)
    ))
fig.show()